In [1]:
from python_prtree import PRTree
import numpy as np
import time
import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 18
import random
import gc
%matplotlib inline

In [2]:
def f(N):
    s = time.time()
    idx = np.arange(N)
    x = np.random.rand(N, 4)
    x[:, 1] = x[:, 0] + x[:, 1] / np.sqrt(N) / 100
    x[:, 3] = x[:, 2] + x[:, 3] / np.sqrt(N) / 100
    prtree = PRTree(idx, x)
    t = time.time()
    x = np.random.rand(100_000, 4)
    x[:, 1] = x[:, 0] + x[:, 1] / np.sqrt(N) / 100
    x[:, 3] = x[:, 2] + x[:, 3] / np.sqrt(N) / 100
    t2 = time.time()
    out = prtree.batch_query(x)
    u = time.time()
    out = [prtree.query(y) for y in x]
    u1 = time.time()
    del_list = random.sample(idx.tolist(), k=min(N, 100_000))
    u2 = time.time()
    for k in del_list:
        prtree.erase(k)
    v = time.time()
    
    x = np.random.rand(min(N, 100_000), 4)
    x[:, 1] = x[:, 0] + x[:, 1] / np.sqrt(N) / 100
    x[:, 3] = x[:, 2] + x[:, 3] / np.sqrt(N) / 100
    v2 = time.time()
    for i, k in enumerate(del_list):
        prtree.insert(k, x[i])
    return t - s, u - t2, (u1 - u), (v - u1), (time.time() - v2)

In [3]:
f(1_000_000)

(0.39927244186401367,
 0.06375265121459961,
 0.5121464729309082,
 0.48781275749206543,
 0.640955924987793)

In [ ]:
x = []
y1 = []
y2 = []
y22 = []
y3 = []
y4 = []
for i in list(range(7, 25))[::-1]:
    n = int(1.5 ** (23 + i))
    print(n, end=', ')
    x.append(n)
    s, t, t2, u,v = f(n)
    y1.append(s)
    y2.append(t)
    y22.append(t2)
    y3.append(u)
    y4.append(v)
    gc.collect()

188924888, 125949925, 83966617, 

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(x, y1)
plt.xscale('log')
plt.xlabel('#rectangles constructed with initial PRTree')
plt.ylabel('time[sec]')
plt.title('construction time')
plt.savefig('./images/fig1.png')

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(x, y2, label='batch_query')
plt.plot(x, y22, label='query')
plt.xscale('log')
plt.xlabel('#rectangles constructed with initial PRTree')
plt.legend()
plt.ylabel('time[sec]')
plt.title('total time of 100,000 queries')
plt.savefig('./images/fig2.png')

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(x, y3, label='erase')
plt.plot(x, y4, label='insert')
plt.xscale('log')
plt.legend()
plt.xlabel('#rectangles constructed with initial PRTree')
plt.ylabel('time[sec]')
plt.title('total time of 100,000 delete/insert events')
plt.savefig('./images/fig3.png')